In [1]:
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import root_mean_squared_error
import os, io
import torch

%matplotlib inline

plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

In [2]:
sample = pd.read_csv('train_umr.csv')

In [3]:
movies = pd.read_csv('movies_encoded.csv')

In [4]:
movies.drop(columns=['Name'],inplace=True)

In [40]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_movies(data,User_ID):
    mov_list=[]
    user_entries = data.loc[data['User_ID'] == User_ID]
    mov_list=user_entries['Movie_ID'].tolist()
    return mov_list

def sim_movies(data, movies, movie_IDs, User_ID):
    sim=np.empty(0)
    sim_scores=np.empty(0)
    for mov in movie_IDs:
        movie=movies.loc[movies['Movie_ID'] == mov]
        sim_matrix = cosine_similarity(movies[['Year', 'isAdult', 'runtimeMinutes', 'movie', 'short',
       'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvSeries', 'tvShort',
       'tvSpecial', 'video', 'Action', 'Adult', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']],movie[['Year', 'isAdult', 'runtimeMinutes', 'movie', 'short',
       'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvSeries', 'tvShort',
       'tvSpecial', 'video', 'Action', 'Adult', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']])
        sim=np.append(sim,sim_matrix.argsort(axis=0)[-50:-1].flatten())
        sim_scores=np.append(sim_scores,np.sort(sim_matrix, axis=0)[-50:-1].flatten())
    
    if len(sim)>=500:
        rec_num=int(len(sim)/50)
    sim_scores =sim_scores.argsort()[-rec_num:]
    out = []
    for i in sim_scores:
            out.append(sim[i])
    return out
    
def retrieve_sim_users(data,sim_ids):
    new_data=pd.DataFrame()
    for i in range(len(sim_ids)):
        a=data.loc[data['Movie_ID'] == sim_ids[i]]
        a=a[['User_ID','Movie_ID','Rating']]
        new_data = pd.concat([new_data,a])
        new_data.drop_duplicates(inplace=True)
        if i%100==0:
            print(i," movies out of",len(sim_ids),"done")
            print("current new data entries:",len(new_data))
    return new_data
 
def sample_new_data(data,movies,User_ID):
    mov_list=retrieve_movies(data,User_ID)
    print(len(mov_list)," retrieved movies for the user")
    print("finding similar movies...")
    sim_mov_list=sim_movies(data,movies,mov_list,User_ID)
    sim_mov_list = np.asarray(sim_mov_list,dtype='int')
    sim_mov_list = np.unique(sim_mov_list)

    for i in range(len(sim_mov_list)):
        cur=sim_mov_list[i]
        if not sample[(sample['User_ID']==User_ID)&(sample['Movie_ID']==cur)].empty:
            sim_mov_list[i]=0
    sim_mov_list=sim_mov_list[sim_mov_list!=0]
    
    print(len(sim_mov_list)," similar unwatched movies found")
    print("retrieving new data")
    new_train_data=retrieve_sim_users(data,sim_mov_list)

    new_test_data=pd.DataFrame(columns=['User_ID','Movie_ID','Rating'])
    
    for i in range(0,len(sim_mov_list)):
        new_test_data.loc[i,"User_ID"]=User_ID
        new_test_data.loc[i,"Movie_ID"]=sim_mov_list[i]
        new_test_data.loc[i,"Rating"]=np.nan
    return new_train_data, new_test_data

In [41]:
train,test = sample_new_data(sample,movies,712664)

317  retrieved movies for the user
finding similar movies...
291  similar unwatched movies found
retrieving new data
0  movies out of 291 done
current new data entries: 0
100  movies out of 291 done
current new data entries: 161502
200  movies out of 291 done
current new data entries: 206294


In [42]:
train

,User_ID,Movie_ID,Rating
385500,1907667,188,3
385501,2411455,188,4
385502,2163521,188,4
385503,1140579,188,3
385504,1103135,188,3
...,...,...,...
13214273,633738,4135,4
13214274,2369921,4135,4
13214275,2603243,4135,2
13214276,2513682,4135,4


In [43]:
test

,User_ID,Movie_ID,Rating
0,712664,29,NaN
1,712664,37,NaN
2,712664,76,NaN
3,712664,185,NaN
4,712664,188,NaN
...,...,...,...
286,712664,10966,NaN
287,712664,10992,NaN
288,712664,11113,NaN
289,712664,11147,NaN
